In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Oct 21 06:15:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!pip install -q tensorflow==2.4.3
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 394.5 MB 351 bytes/s 
     |████████████████████████████████| 2.9 MB 52.4 MB/s 
     |████████████████████████████████| 462 kB 58.2 MB/s 
     |████████████████████████████████| 3.8 MB 49.2 MB/s 
     |████████████████████████████████| 591 kB 5.1 MB/s 
     |████████████████████████████████| 120 kB 64.8 MB/s 
     |████████████████████████████████| 1.1 MB 45.7 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
     |████████████████████████████████| 103 kB 51.6 MB/s 
     |████████████████████████████████| 87 kB 7.0 MB/s 
     |████████████████████████████████| 213 kB 61.8 MB/s 
     |████████████████████████████████| 840 kB 57.4 MB/s 
     |████████████████████████████████| 6.3 MB 24.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.6 MB/s 
     |████████████████████████████████| 1.1 MB 52.0 MB/s 
     |████████████████████████████████| 47.6 MB 68 kB/s 
     |██████████

In [4]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [5]:
spec = model_spec.get('efficientdet_lite3')

In [6]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('/content/gdrive/My Drive/ColabNotebooks/allData_0930.csv')

In [7]:
model = object_detector.create(train_data, epochs=100, model_spec=spec, batch_size=16, train_whole_model=True, validation_data=validation_data)

Epoch 1/100
98/98 [==============================] - 185s 1s/step - det_loss: 1.4178 - cls_loss: 1.0592 - box_loss: 0.0072 - reg_l2_loss: 0.0940 - loss: 1.5118 - learning_rate: 0.0140 - gradient_norm: 2.2970 - val_det_loss: 1.2283 - val_cls_loss: 0.8869 - val_box_loss: 0.0068 - val_reg_l2_loss: 0.0942 - val_loss: 1.3225
Epoch 2/100
98/98 [==============================] - 109s 1s/step - det_loss: 0.8105 - cls_loss: 0.6270 - box_loss: 0.0037 - reg_l2_loss: 0.0947 - loss: 0.9052 - learning_rate: 0.0200 - gradient_norm: 3.5050 - val_det_loss: 0.8745 - val_cls_loss: 0.6050 - val_box_loss: 0.0054 - val_reg_l2_loss: 0.0951 - val_loss: 0.9696
Epoch 3/100
98/98 [==============================] - 109s 1s/step - det_loss: 0.6425 - cls_loss: 0.4860 - box_loss: 0.0031 - reg_l2_loss: 0.0954 - loss: 0.7379 - learning_rate: 0.0200 - gradient_norm: 2.9140 - val_det_loss: 0.7353 - val_cls_loss: 0.4693 - val_box_loss: 0.0053 - val_reg_l2_loss: 0.0957 - val_loss: 0.8310
Epoch 4/100
98/98 [===============

In [8]:
model.summary()

Model: ""
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  8315968   
_________________________________________________________________
class_net/class-predict (Sep multiple                  15930     
_________________________________________________________________
box_net/box-predict (Separab multiple                  7236      
Total params: 8,339,134
Trainable params: 8,234,942
Non-trainable params: 104,192
_________________________________________________________________


In [9]:
model.export(export_dir='/content/gdrive/My Drive/ColabNotebooks/modelDir')

In [ ]:
#model.evaluate(test_data)

In [ ]:
model.evaluate_tflite('/content/gdrive/My Drive/ColabNotebooks/modelDir/model.tflite', test_data)